<a href="https://colab.research.google.com/github/plk456/bs4/blob/main/strokeprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
n=pd.read_csv('healthcare-dataset-stroke-data.csv')
print(n.head())

      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
from sklearn.preprocessing import LabelEncoder
le_gender=LabelEncoder()
le_ever_married=LabelEncoder()
le_work_type=LabelEncoder()
le_residence=LabelEncoder()
le_smoking=LabelEncoder()


le_gender.fit(n['gender'])
le_ever_married.fit(n['ever_married'])
le_work_type.fit(n['work_type'])
le_residence.fit(n['Residence_type'])
le_smoking.fit(n['smoking_status'])


n['genders']=le_gender.transform(n['gender'])
n['marriage_status']=le_ever_married.transform(n['ever_married'])
n['work']=le_work_type.transform(n['work_type'])
n['Residence']=le_residence.transform(n['Residence_type'])
n['smoker_or_not']=le_smoking.transform(n['smoking_status'])

In [ ]:
n=n.drop(['gender','work_type','Residence_type','smoking_status'],axis=1)
n=n.drop(['ever_married'],axis=1)

In [ ]:
print(n)

         id   age  hypertension  heart_disease  avg_glucose_level   bmi  \
0      9046  67.0             0              1             228.69  36.6   
1     51676  61.0             0              0             202.21   NaN   
2     31112  80.0             0              1             105.92  32.5   
3     60182  49.0             0              0             171.23  34.4   
4      1665  79.0             1              0             174.12  24.0   
...     ...   ...           ...            ...                ...   ...   
5105  18234  80.0             1              0              83.75   NaN   
5106  44873  81.0             0              0             125.20  40.0   
5107  19723  35.0             0              0              82.99  30.6   
5108  37544  51.0             0              0             166.29  25.6   
5109  44679  44.0             0              0              85.28  26.2   

      stroke  genders  marriage_status  work  Residence  smoker_or_not  
0          1        1     

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
imputer.fit(n.iloc[:,5:6])
n.iloc[:,5:6]=imputer.transform(n.iloc[:,5:6])


In [ ]:
print(n)

         id   age  hypertension  heart_disease  avg_glucose_level        bmi  \
0      9046  67.0             0              1             228.69  36.600000   
1     51676  61.0             0              0             202.21  28.893237   
2     31112  80.0             0              1             105.92  32.500000   
3     60182  49.0             0              0             171.23  34.400000   
4      1665  79.0             1              0             174.12  24.000000   
...     ...   ...           ...            ...                ...        ...   
5105  18234  80.0             1              0              83.75  28.893237   
5106  44873  81.0             0              0             125.20  40.000000   
5107  19723  35.0             0              0              82.99  30.600000   
5108  37544  51.0             0              0             166.29  25.600000   
5109  44679  44.0             0              0              85.28  26.200000   

      stroke  genders  marriage_status 

In [ ]:
from sklearn.model_selection import train_test_split
x=n.drop(['stroke'],axis=1)
X=x.head(500)
y=n['stroke']
Y=y.head(500)
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
#print(y_pred.reshape(-1,1))

[[1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]]


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7


In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[35 16]
 [14 35]]


In [ ]:
print(x_test)

      age  hypertension  heart_disease  avg_glucose_level   bmi  genders  \
90   79.0             0              1             226.98  29.8        0   
254  47.0             0              0             210.95  50.1        0   
283  58.0             0              0             149.75  27.0        0   
445  13.0             0              0              55.39  23.2        1   
461  16.0             0              0             113.47  19.5        0   
..    ...           ...            ...                ...   ...      ...   
372  41.0             0              0              99.80  31.6        1   
56   82.0             0              0              59.32  33.2        0   
440  22.0             0              0             130.34  22.0        0   
60   56.0             0              0             185.17  40.4        0   
208  78.0             1              0             134.80  33.6        1   

     marriage_status  work  Residence  smoker_or_not  
90                 1     2      

In [ ]:
w=[32,0,0,74,23,0,0,2,1,3]
w=np.array(w)
w=w.reshape(1,-1)
w_pred=model.predict(w)
print(w_pred)

[0]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
